## Flight Price Prediction EDA and Feature Engineering

Data sets:
https://github.com/krishnaik06/5-Days-Live-EDA-and-Feature-Engineering/tree/main/Flight%20Predictionhttps://github.com/krishnaik06/5-Days-Live-EDA-and-Feature-Engineering/tree/main/Flight%20Prediction

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import os
import re
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

### Load train dataset

In [2]:
df = pd.read_excel('Data_Train.xlsx')
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


### Load test dataset

In [3]:
df2 = pd.read_excel('Test_set.xlsx')
df2.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


### Make a copy of loaded datasets

In [4]:
train_df = df.copy()
test_df = df2.copy()

### Feature engineering functions

In [5]:
# Global variables
enc = OneHotEncoder(drop='first', dtype='int16', handle_unknown='ignore')
total_stops_dict = {'non-stop':0,'1 stop':1,'2 stops':2, '3 stops':3, '4 stops':4, np.nan:1}

# old column: 24/03/2019 -> nem columns: date:24, month:3, year:2019
def separate_date(df, column):    
    for i,col in enumerate(['Day','Month','Year']):
        df[col] = df[column].map(lambda x: int(x.split('/')[i]))
    df.drop(column, axis=1, inplace=True)
    return df

# 01:10 22 Mar -> 01:10
def extract_time(df, column):
    df[column] = df[column].map(lambda x: x.split(' ')[0])
    return df
    
# 22:20 -> new columns: hour:22, min:20
def extract_hour_min(df, column):
    prefix= column.split('_')[0]
    hour, Min = prefix+'_Hour', prefix+'_Min'
    df[hour] = df[column].map(lambda x: int(x.split(':')[0]))
    df[Min] = df[column].map(lambda x: int(x.split(':')[1]))
    df.drop(column, axis=1, inplace=True)
    return df

# Total_Stops {'non-stop':0,'1 stop':1,'2 stops':2, '3 stops':3, '4 stops':4, np.nan:1}
def replace_values(df, column, val_dict):
    df[column] = df[column].map(val_dict)
    return df

# ['Route']
def drop_redundant(df, columns_lst):
    df.drop(columns_lst, axis=1, inplace=True)
    return df

# Duration 5m -> 4h 15m
def replace_with_correct_value(df, column, old_val, new_val):
    df[column] = df[column].replace({old_val:new_val})
    return df

# 2h 50m -> 170
def conver_h_m_into_min(row):
    delimeters = 'h', 'm', ' '
    regexPattern = '|'.join(map(re.escape, delimeters))
    row = re.split(regexPattern, row)    
    h = int(row[0])
    try:
        m = int(row[2])
    except:
        m = 0        
    return 60*h+m

# Create new column for time in min: 170 <- 2h 50m
def convert_time(df, column):
    prefix = column.split('_')[0]
    new_col = prefix+'_in_Min'
    df[new_col] = df[column].apply(lambda x: conver_h_m_into_min(x))
    df.drop(column, axis=1, inplace=True)
    return df

# Encode categorical features and combine with rest DataFrame
def encode_1_hot(df, columns_lst, encoder, test=False):
    if not test:
        encoder.fit(df[columns_lst])
    enc_df = encoder.transform(df[columns_lst])
    
    features = encoder.get_feature_names_out(columns_lst)
    enc_df = pd.DataFrame(enc_df.todense(), columns=features)
    
    df = pd.concat([df, enc_df], axis=1)
    df.drop(columns_lst, axis=1, inplace=True)
    return df

### Reduce memory usage

In [6]:
def reduce_memory_usage(df):
    # import external ipynb file function
    from ipynb.fs.defs.reduce_memory import external_rmu
    
    return external_rmu(df)

### Preprocessing function

In [7]:
def preprocess_dataset(df, test=False):
    # Split Date_of_Journey feature into 3 separate: <Date, Month, Year>
    df = separate_date(df, 'Date_of_Journey')
    
    # Extract from Arrival_time time: <01:10 22 Mar -> 01:10>
    df = extract_time(df, 'Arrival_Time')
    
    # Extract into new columns hours/min from Arrival/Departure: <22:20 -> new columns: Hour:22, Min:20>
    for col in ['Arrival_Time', 'Dep_Time']:
        df = extract_hour_min(df, col)
    
    # Convert categorical into int
    df = replace_values(df, 'Total_Stops', total_stops_dict)
    
    # Route redundant as number of stops exist
    df = drop_redundant(df, ['Route'])
    
    # Replace incorrect flight duration
    df = replace_with_correct_value(df, 'Duration', '5m', '4h 15m')
    
    # Conver time in h and m into min: 2h 50m -> 170
    df = convert_time(df, 'Duration')
    
    # Encode categorical variables with binary dummies by OneHotEncoder
    categorical = ['Airline', 'Source', 'Destination', 'Additional_Info']
    df = encode_1_hot(df, categorical, enc, test=test)
    
    # Reduce memory usage to speed up processing data
    reduce_memory_usage(df)
    
    return df

### Explore dataset

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


### Check missing values

In [9]:
train_df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

### Total_Stops feature

In [10]:
train_df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [11]:
train_df[train_df['Total_Stops'].isnull()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


In [12]:
similar_flight = train_df.loc[((train_df['Source']=='Delhi') & (train_df['Destination']=='Cochin'))]
similar_flight

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
9,Multiple carriers,27/05/2019,Delhi,Cochin,DEL → BOM → COK,11:25,19:15,7h 50m,1 stop,No info,8625
10,Air India,1/06/2019,Delhi,Cochin,DEL → BLR → COK,09:45,23:00,13h 15m,1 stop,No info,8907
15,Air India,3/03/2019,Delhi,Cochin,DEL → AMD → BOM → COK,16:40,19:15 04 Mar,26h 35m,2 stops,No info,14011
16,SpiceJet,15/04/2019,Delhi,Cochin,DEL → PNQ → COK,08:45,13:15,4h 30m,1 stop,No info,5830
...,...,...,...,...,...,...,...,...,...,...,...
10669,Air India,15/06/2019,Delhi,Cochin,DEL → BOM → COK,08:00,19:15,11h 15m,1 stop,No info,9929
10672,Jet Airways,27/06/2019,Delhi,Cochin,DEL → AMD → BOM → COK,23:05,19:00 28 Jun,19h 55m,2 stops,In-flight meal not included,11150
10673,Jet Airways,27/05/2019,Delhi,Cochin,DEL → AMD → BOM → COK,13:25,04:25 28 May,15h,2 stops,No info,16704
10676,Multiple carriers,1/05/2019,Delhi,Cochin,DEL → BOM → COK,10:20,19:00,8h 40m,1 stop,No info,9794


In [13]:
similar_flight['Total_Stops'].mode()

0    1 stop
dtype: object

### Extract hours and min for Duration column

In [14]:
# Check false values of Duration
new_df = train_df['Duration'].value_counts().to_frame().reset_index().rename(columns={'index':'Duration','Duration':'freq'})
new_df['len'] = new_df['Duration'].map(lambda x: len(x))
new_df.sort_values(by=['len']).head(10)

,Duration,freq,len
350,5m,1,2
26,5h,63,2
159,7h,21,2
22,8h,68,2
122,6h,26,2
24,9h,64,2
227,4h,12,2
5,3h,261,2
117,24h,27,3
91,27h,32,3


In [15]:
# Find row with false value of Duration
train_df[train_df['Duration']=='5m']

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


In [16]:
# Find similar flights
similar = train_df.loc[((train_df['Source']=='Mumbai') & 
                        (train_df['Destination']=='Hyderabad'))]
similar
# Value 5309 accordin to all filters and price seems to be the same flight and it took 4h 15m

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
59,Vistara,6/03/2019,Mumbai,Hyderabad,BOM → DEL → HYD,15:40,16:45 07 Mar,25h 5m,1 stop,No info,12395
67,Air India,6/05/2019,Mumbai,Hyderabad,BOM → HYD,13:55,15:25,1h 30m,non-stop,No info,3625
70,Jet Airways,1/06/2019,Mumbai,Hyderabad,BOM → HYD,07:05,08:30,1h 25m,non-stop,No info,5678
77,IndiGo,3/04/2019,Mumbai,Hyderabad,BOM → HYD,06:25,07:55,1h 30m,non-stop,No info,4049
98,Jet Airways,18/06/2019,Mumbai,Hyderabad,BOM → HYD,07:05,08:30,1h 25m,non-stop,No info,8040
...,...,...,...,...,...,...,...,...,...,...,...
10611,Jet Airways,18/05/2019,Mumbai,Hyderabad,BOM → HYD,07:05,08:30,1h 25m,non-stop,In-flight meal not included,4050
10617,IndiGo,27/05/2019,Mumbai,Hyderabad,BOM → HYD,02:35,04:05,1h 30m,non-stop,No info,2754
10655,Jet Airways,3/05/2019,Mumbai,Hyderabad,BOM → HYD,02:55,04:20,1h 25m,non-stop,No info,5678
10671,Air India,6/06/2019,Mumbai,Hyderabad,BOM → HYD,21:05,22:25,1h 20m,non-stop,No info,3100


### Check unique values for other categorical features

In [17]:
for col in ['Airline', 'Source', 'Destination', 'Additional_Info']:
    print(col, 'unique values:', '\n', train_df[col].unique(), end='\n\n')

Airline unique values: 
 ['IndiGo' 'Air India' 'Jet Airways' 'SpiceJet' 'Multiple carriers' 'GoAir'
 'Vistara' 'Air Asia' 'Vistara Premium economy' 'Jet Airways Business'
 'Multiple carriers Premium economy' 'Trujet']

Source unique values: 
 ['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai']

Destination unique values: 
 ['New Delhi' 'Banglore' 'Cochin' 'Kolkata' 'Delhi' 'Hyderabad']

Additional_Info unique values: 
 ['No info' 'In-flight meal not included' 'No check-in baggage included'
 '1 Short layover' 'No Info' '1 Long layover' 'Change airports'
 'Business class' 'Red-eye flight' '2 Long layover']



### Preprocess datasets

In [18]:
train_df = preprocess_dataset(train_df)
train_df

Mem. usage decreased to 0.45 Mb (68.1% reduction)


,Total_Stops,Price,Day,Month,Year,Arrival_Hour,Arrival_Min,Dep_Hour,Dep_Min,Duration_in_Min,...,Destination_New Delhi,Additional_Info_1 Short layover,Additional_Info_2 Long layover,Additional_Info_Business class,Additional_Info_Change airports,Additional_Info_In-flight meal not included,Additional_Info_No Info,Additional_Info_No check-in baggage included,Additional_Info_No info,Additional_Info_Red-eye flight
0,0,3897,24,3,2019,1,10,22,20,170,...,1,0,0,0,0,0,0,0,1,0
1,2,7662,1,5,2019,13,15,5,50,445,...,0,0,0,0,0,0,0,0,1,0
2,2,13882,9,6,2019,4,25,9,25,1140,...,0,0,0,0,0,0,0,0,1,0
3,1,6218,12,5,2019,23,30,18,5,325,...,0,0,0,0,0,0,0,0,1,0
4,1,13302,1,3,2019,21,35,16,50,285,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,0,4107,9,4,2019,22,25,19,55,150,...,0,0,0,0,0,0,0,0,1,0
10679,0,4145,27,4,2019,23,20,20,45,155,...,0,0,0,0,0,0,0,0,1,0
10680,0,7229,27,4,2019,11,20,8,20,180,...,0,0,0,0,0,0,0,0,1,0
10681,0,12648,1,3,2019,14,10,11,30,160,...,1,0,0,0,0,0,0,0,1,0


In [19]:
test_df = preprocess_dataset(test_df, test=True)
test_df

Mem. usage decreased to 0.10 Mb (69.2% reduction)


,Total_Stops,Day,Month,Year,Arrival_Hour,Arrival_Min,Dep_Hour,Dep_Min,Duration_in_Min,Airline_Air India,...,Destination_New Delhi,Additional_Info_1 Short layover,Additional_Info_2 Long layover,Additional_Info_Business class,Additional_Info_Change airports,Additional_Info_In-flight meal not included,Additional_Info_No Info,Additional_Info_No check-in baggage included,Additional_Info_No info,Additional_Info_Red-eye flight
0,1,6,6,2019,4,25,17,30,655,0,...,0,0,0,0,0,0,0,0,1,0
1,1,12,5,2019,10,20,6,20,240,0,...,0,0,0,0,0,0,0,0,1,0
2,1,21,5,2019,19,0,19,15,1425,0,...,0,0,0,0,0,1,0,0,0,0
3,1,21,5,2019,21,0,8,0,780,0,...,0,0,0,0,0,0,0,0,1,0
4,0,24,6,2019,2,45,23,55,170,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,1,6,6,2019,20,25,20,30,1435,1,...,0,0,0,0,0,0,0,0,1,0
2667,0,27,3,2019,16,55,14,20,155,0,...,0,0,0,0,0,0,0,0,1,0
2668,1,6,3,2019,4,25,21,50,395,0,...,0,0,0,0,0,0,0,0,1,0
2669,1,6,3,2019,19,15,4,0,915,1,...,0,0,0,0,0,0,0,0,1,0
